## Introduccion

### Descripcion del proyecto

Este proyecto surge como una solución práctica a un desafío actual de Megaline, una compañía de telecomunicaciones. Con el avance constante en los servicios móviles, Megaline se ha dado cuenta de que muchos de sus clientes todavía utilizan planes antiguos. Por ello, el proyecto se centtra en desarrollar un método que nos permita analizar los hábitos de uso de los clientes y, en base a esto, recomendar a los clientes uno de los planes más modernos de la empresa: Smart o Ultra. La idea es hacer uso de los datos disponibles sobre los usuarios para hacer recomendaciones personalizadas que se alineen con sus patrones de consumo.

### Objetivos

Desarrollar un Modelo Predictivo: Crear un modelo que pueda identificar el plan más adecuado para cada cliente, utilizando datos de su comportamiento pasado.

Preparación y Análisis de Datos: Examinar y procesar el conjunto de datos proporcionado para asegurar su idoneidad para el modelo.

División Estratégica de Datos: Segmentar correctamente los datos en conjuntos de entrenamiento, validación y prueba, garantizando así la efectividad y precisión del modelo.

## Importacion de librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Exploración y Preparación de Datos

In [3]:
df = pd.read_csv('datasets/users_behavior.csv')
print(df.head())
print()
print(df.info())
print()
print(df.dtypes)

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

calls       float64
minutes     float64
messages    float64
mb_used     float64
is_ultra      int64
dtype: object


Al ver los datos podemos darnos cuenta que no existen datos nulos en nuestro dataset, pero si podemos ver que los mensajes y llamadas son del tipo float, podria ser conveniente modificarlos a int ya que sus valores son enteros.

### Modificacion de tipo de datos

In [ ]:
df['calls'] = df['calls'].astype(int)
df['messages'] = df['messages'].astype(int)

### Verificacion de tipo de datos

In [ ]:
print(df.dtypes)
print()
print(df.head())

calls         int64
minutes     float64
messages      int64
mb_used     float64
is_ultra      int64
dtype: object

   calls  minutes  messages   mb_used  is_ultra
0     40   311.90        83  19915.42         0
1     85   516.75        56  22696.96         0
2     77   467.66        86  21060.45         0
3    106   745.53        81   8437.39         1
4     66   418.74         1  14502.75         0


### Busqueda de valores duplicados

In [ ]:
duplicados_df = df.duplicated()
print(f"Duplicados encontrados en df = {duplicados_df.sum()}")

Duplicados encontrados en df = 0


## División de Datos en Conjuntos de Entrenamiento, Validación y Prueba


In [ ]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_temp, target_train, target_temp = train_test_split(
    features, target, test_size=0.40, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_temp, target_temp, test_size=0.50, random_state=12345)

## Analisis de modelos

### Desarrollo y Evaluación del Modelo de Regresión Logistica




In [ ]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)

predictions_valid = model.predict(features_valid)

result = accuracy_score(target_valid, predictions_valid)
print("Precisión del modelo de regresión logística en el conjunto de validación:", result)

predictions_test = model.predict(features_test)

result_test = accuracy_score(target_test, predictions_test)
print("Precisión del modelo de regresión logística en el conjunto de prueba:", result_test)


Precisión del modelo de regresión logística en el conjunto de validación: 0.7107309486780715
Precisión del modelo de regresión logística en el conjunto de prueba: 0.6842923794712286


### Desarrollo y Evaluación del Modelo de Bosque Aleatorio



In [ ]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(max_depth=depth, random_state=12345, n_estimators=est)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("Precisión del mejor modelo en el conjunto de validación:", best_result, "n_estimators:", best_est, "best_depth:", best_depth)

predictions_test = best_model.predict(features_test)

result_test = accuracy_score(target_test, predictions_test)
print("Precisión del Modelo de Bosque Aleatorio en el conjunto de prueba:", result_test)

Precisión del mejor modelo en el conjunto de validación: 0.8087091757387247 n_estimators: 40 best_depth: 8
Precisión del Modelo de Bosque Aleatorio en el conjunto de prueba: 0.7962674961119751


### Desarrollo y Evaluación del Modelo de Arbol de decision de regresion


In [ ]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

print(f"Precisión del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")

predictions_test = best_model.predict(features_test)

result_test = accuracy_score(target_test, predictions_test)
print("Precisión del Modelo de Arbol de decision de regresion en el conjunto de prueba:", result_test)

Precisión del mejor modelo en el conjunto de validación (max_depth = 3): 0.7853810264385692
Precisión del Modelo de Arbol de decision de regresion en el conjunto de prueba: 0.7791601866251944


## Comparación y Selección del Mejor Modelo



Después de experimentar con varios modelos, los resultados son claros y bastante reveladores. Iniciamos con el modelo de regresión logística, que nos dio una precisión decente de alrededor del 71.0%, un buen punto de inicio pero no lo suficiente para el objetivo del proyecto. Luego, con el modelo de bosque aleatorio, ajustando los hiperparámetros, conseguimos una precisión del 80.8% con 40 estimadores y una profundidad máxima de 8 del conjunto de validacion, y también mantuvo un alto rendimiento con una precisión del 79.6% en el conjunto de prueba.  Este resultado no solo superó nuestras expectativas, sino que también le gano al modelo de árbol de decisión, que con un max_depth de 3, logró una precisión del 78.5% en el conjunto de validacion y un 77.9% en el conjunto de prueba. 

El modelo de bosque aleatorio demostro ser el más adecuado para entender y predecir el comportamiento de nuestros usuarios. Con esto, decidimos que el bosque aleatorio con sus parámetros afinados es nuestra mejor apuesta para la implementación.

## Conclusiones y Recomendaciones



Al finalizar este proyecto, hemos obtenido insights valiosos y aprendizajes significativos. Lo más destacado ha sido comprobar la viabilidad de usar el análisis de datos para predecir de forma precisa el plan de telefonía móvil más adecuado para los clientes. Entre los diversos modelos que exploramos, el Bosque Aleatorio, con su  precisión del 80.8%, gano como la herramienta más prometedora.

Este hallazgo es relevante, ya que destaca la importancia de adaptar y refinar constantemente las estrategias de análisis de datos para mantenerse al día con las necesidades cambiantes de los clientes. 

